# Support for Quantum ESPRESSO in exatomic

In [1]:
import os
import exatomic
from exatomic import qe


def get_path(name):
    """Helper function for building paths"""
    return os.path.abspath(os.path.join(os.path.abspath(exatomic.__file__), "../static/qe/qe."+name+".bz2"))

In [2]:
#path = os.path.abspath(os.path.join(os.path.abspath(exatomic.__file__), "../static/qe/qe.inp.bz2"))
symbols = qe.parse_symbols_from_input(get_path("inp"))

In [3]:
pos = qe.parse_xyz(get_path("pos"), symbols)
vel = qe.parse_xyz(get_path("vel"), symbols)
force = qe.parse_xyz(get_path("for"), symbols)

In [4]:
evp = qe.parse_evp(get_path("evp"), symbols)

In [5]:
pos.head()

,x,y,z,frame,symbol
0,1.386029,10.148803,2.527119,86300,H
1,6.298432,0.408980,6.614086,86300,H
2,2.705972,4.812221,1.851418,86300,H
3,0.925476,7.572369,3.711328,86300,H
4,-0.712621,7.029148,6.158907,86300,H
